In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import functools
import random
import time
from threading import Thread, Condition, Lock
from typing import List

In [2]:
def generate_matrix(length: int):
    matrix = np.random.randint(1, 1000, size=(length, length))
    matrix_symmetric = (matrix + matrix.T) / 2
    for i in range(length):
        matrix_symmetric[i][i] = None
    return matrix_symmetric

def print_graph(matrix):
    # Create DiGraph from A
    graph = nx.from_numpy_matrix(matrix, create_using=nx.DiGraph())

    # Use spring_layout to handle positioning of graph
    layout = nx.spring_layout(graph)

    # Use a list for node_sizes
    sizes = []

    # Use a list for node colours
    color_map = []

    for i in range(len(matrix[0])):
        sizes.append(200)
        value = i % 9 + 1
        if value == 1:
            color_map.append('green')
        elif value == 2:
            color_map.append('red')
        elif value == 3:
            color_map.append('blue')
        elif value == 4:
            color_map.append('cyan')
        elif value == 5:
            color_map.append('brown')
        elif value == 6:
            color_map.append('black')
        elif value == 7:
            color_map.append('lime')
        elif value == 8:
            color_map.append('m')
        elif value == 9:
            color_map.append('y')

    # Draw the graph using the layout - with_labels=True if you want node labels.
    nx.draw(graph, layout, with_labels=True, node_size=sizes, node_color=color_map)

    # Get weights of each edge and assign to labels
    labels = nx.get_edge_attributes(graph, "weight")

    # Draw edge labels using layout and list of labels
    nx.draw_networkx_edge_labels(graph, pos=layout, edge_labels=labels)

    # Show plot
    plt.show()

In [87]:
graph = generate_matrix(20)
#print_graph(graph)

In [88]:
print(graph)

[[  nan 558.  449.  418.  336.  611.5 117.5 588.  606.  345.  729.5 198.5
  566.  534.  507.5 659.5 790.   66.5 198.  504.5]
 [558.    nan 470.5 511.  501.5 401.  730.  440.5  75.5 665.5 519.5 108.
  264.5 519.  569.  594.5 326.  345.  608.  588.5]
 [449.  470.5   nan 820.  485.  369.5 574.  353.5 774.  181.  432.5 466.
  170.5 652.5 398.  364.  526.  439.5 381.5 701.5]
 [418.  511.  820.    nan 377.  782.  736.  353.  655.  470.  465.5 644.
  703.  153.5 366.5 724.5 246.5 744.  340.5 293.5]
 [336.  501.5 485.  377.    nan 606.  483.5 401.  902.  261.5 509.5 914.
  150.  476.  299.5 508.5 706.  328.5 615.5 694.5]
 [611.5 401.  369.5 782.  606.    nan 435.5 357.  490.  467.  361.  671.
  619.  734.  491.  347.5 581.5 497.5 288.5 733. ]
 [117.5 730.  574.  736.  483.5 435.5   nan 625.5 315.5 446.  345.5 305.
  612.5 557.5 408.5 479.5 475.  694.5 696.5 225.5]
 [588.  440.5 353.5 353.  401.  357.  625.5   nan 920.5 588.  337.5 307.
  575.5 660.  156.5 420.5 611.5 795.5 720.5 294.5]
 [606. 

In [104]:
#Impact of pheromones
ALPHA = 0.5
#Impact of weights
BETA = 0.5

class Ant(Thread):
    def __init__(self, init_pos: int, graph: List[List[float]], pheromones: List[List[float]], callback) -> None:
        Thread.__init__(self)
        self.init_pos = init_pos
        self.graph = graph
        self.has_arrived = False
        self.callback = callback
        self.wait_cond = Condition(Lock())
        self.reset(pheromones)

    def reset(self, pheromones: List[List[float]]) -> None:
        self.pheromones = pheromones
        self.current_pos = self.init_pos
        self.path_history = [self.init_pos]
        if self.has_arrived:
            self.has_arrived = False
            self.wait_cond.notify()
            self.wait_cond.release()

    def wait_for_reset(self) -> None:
        self.has_arrived = True
        self.wait_cond.acquire()
        
    def run(self):
        while True:
            with self.wait_cond:
                while self.has_arrived:
                    self.wait_cond.wait()
            self.move(self.graph[self.current_pos], self.pheromones[self.current_pos])

    def move(self, dest: list, pheromones: list) -> None:
        probabilities = []
        denominator = 0
        
        #Calculate the denominator first
        for i in range(len(dest)):
            if not np.isnan(dest[i]) and i not in self.path_history:
                denominator += pheromones[i]**ALPHA * (1/dest[i])**BETA

        #Calculate probabilities of picking one path
        for node, length in enumerate(dest):
            if node in self.path_history:
                probabilities.append(0)
            elif not np.isnan(dest[node]):
                nominator = pheromones[node]**ALPHA * (1/dest[node])**BETA
                probabilities.append(nominator / denominator)
            else:
                probabilities.append(np.nan)
        
        #If there is no path available return false
        if np.nansum(probabilities) == 0:
            self.path_history.append(self.init_pos)
            self.wait_for_reset()
            self.callback(self.path_history)
            return

        #Roulette wheel
        cumulative_sum = []
        for i in range(len(probabilities)):
            if np.isnan(probabilities[i]):
                cumulative_sum.append(np.nan)
            elif np.nansum(cumulative_sum) == 0:
                cumulative_sum.append(1.0)
            else:
                cumulative_sum.append(np.nansum(probabilities[i:len(probabilities)]))
        
        #Pick a destination
        rand = random.uniform(0, 1)
        dest_picked = None
        cumulative_sum.append(0)

        for i in range(0, len(cumulative_sum)-1):
            p = cumulative_sum[i]
            nextp = cumulative_sum[i+1] if not np.isnan(cumulative_sum[i+1]) else cumulative_sum[i+2] if not np.isnan(cumulative_sum[i+2]) else cumulative_sum[i+3]
        
            if not np.isnan(p) and rand <= p and rand >= nextp:
                dest_picked = i

        self.current_pos = dest_picked
        self.path_history.append(self.current_pos)

In [105]:
RHO = 0.5
Q = 1

class ACO():
    def __init__(self, graph, start: int) -> None:
        self.graph = graph
        self.start = start
        self.pheromones = np.ones(graph.shape)
        self.ants_running = 0
        self.current_paths = []
        self.best_path = None
        self.lock = Lock()

    def run(self, amount_iterations: int, ants_amount: int, done_callback) -> None:
        ants = [Ant(self.start, self.graph, self.pheromones, self.print_callback) for i in range(ants_amount)]
        self.ants_running = ants_amount
        for ant in ants:
            ant.start()
        for i in range(amount_iterations):
            while self.ants_running != 0:
                pass
            with self.lock:
                self.ants_running = ants_amount
                self.update_pheromones(self.current_paths)
                if i == amount_iterations - 1:
                    self.best_path = self.get_final_path(self.current_paths)
                    break
                self.current_paths = []
                print('iteration:', i)
                for ant in ants:
                    ant.reset(self.pheromones)
        return done_callback(self.best_path)
    
    def print_callback(self, path_history):
        self.ants_running -= 1
        self.current_paths.append(path_history)

    def update_pheromones(self, paths: list) -> None:
        #Evaporation
        self.pheromones *= 1 - RHO
        #New pheromones
        for path in paths:
            total_distance = self.calc_total_distance(path)
            for i in range(len(path)-1):
                cur_path = path[i]
                next_path = path[i+1]
                self.pheromones[cur_path][next_path] += Q / total_distance
                self.pheromones[next_path][cur_path] += Q / total_distance

    def calc_total_distance(self, full_path: list) -> float:
        distance = 0
        for i in range(len(full_path)-1):
            cur_path = full_path[i]
            next_path = full_path[i+1]
            distance += self.graph[cur_path][next_path]
        return distance

    def get_final_path(self, paths: list) -> list:
        best_path = paths[0]
        lowest_distance = self.calc_total_distance(paths[0])
        for p in paths:
            distance = self.calc_total_distance(p)
            if distance < lowest_distance:
                best_path = p
                lowest_distance = distance
        print(lowest_distance)
        return best_path


In [106]:
aco = ACO(graph, 0)

def done_callback(best_path):
    print('time:', time.time()-start)
    print('done', best_path)
    
start = time.time()
path = aco.run(30, 50, done_callback)

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 20
iteration: 21
iteration: 22
iteration: 23
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
5879.0
time: 40.36976647377014
done [0, 4, 9, 10, 5, 14, 3, 13, 8, 6, 19, 15, 17, 18, 11, 16, 12, 2, 7, 1, 0]
